In [3]:
import numpy as np
from qiskit import *
from qiskit.circuit import Parameter
import qiskit.quantum_info as qi

In [9]:
from qiskit import IBMQ
from IBM_Q import API_IBM

IBMQ.save_account(API_IBM, overwrite=True)


IBMQ.load_account() # Load account from disk
IBMQ.providers()    # List all available providers

ibmqfactory.load_account:WARNING:2021-05-19 22:13:57,211: Credentials are already in use. The existing account in the session will be replaced.


In [14]:
IBMQ.providers() 

[<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>,
 <AccountProvider for IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>]

In [15]:
provider = IBMQ.get_provider(hub='ibm-q-utokyo')

In [16]:
provider.backends()

[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmqx2') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_16_melbourne') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_athens') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_rome') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-utokyo', group='internal', project='qc2021s')>,
 <IBMQ

In [18]:
backend = provider.get_backend('ibmq_montreal')

In [13]:
# provider.backends('ibmq_sydney')

[]

In [19]:
q = QuantumRegister(3)
c = ClassicalRegister(2)

In [20]:
u_circuit = QuantumCircuit(q,c, name='teleportation')


theta = Parameter('θ')
phi = Parameter('φ')
psy = Parameter('ψ')


u_circuit.u(theta,phi,psy,q[0])
u_circuit.barrier(q)

qc = QuantumCircuit(q,c, name='teleportation')


qc.h(q[1])
qc.cnot(q[1],q[2])
qc.x(q[1])
qc.barrier(q)

qc.cnot(q[0],q[1])
qc.h(q[0])
qc.barrier(q)

qc.measure([q[0],q[1]],[c[0],c[1]])


qc.x(q[2]).c_if(c, 0)
qc.z(q[2]).c_if(c, 3)
qc.z(q[2]).c_if(c, 2)
qc.x(q[2]).c_if(c, 2)

In [21]:
T = np.random.rand(3) * np.pi
parameters = {theta:T[0], phi:T[1], psy:T[2]}
u_circuit_ = u_circuit.bind_parameters(parameters)

In [8]:
backend = BasicAer.get_backend('statevector_simulator')
job = execute(u_circuit_, backend)
qc_state = qi.Statevector(job.result().get_statevector(u_circuit_))
qi.partial_trace(qc_state,[1,2])

DensityMatrix([[ 0.3118348 +0.j        , -0.00761719-0.46318013j],
               [-0.00761719+0.46318013j,  0.6881652 +0.j        ]],
              dims=(2,))


In [9]:
original_qc = u_circuit.combine(qc)
original_qc.draw('mpl')

original_qc_ = original_qc.bind_parameters(parameters)

backend = BasicAer.get_backend('statevector_simulator')
job = execute(original_qc_, backend)
qc_state = qi.Statevector(job.result().get_statevector(original_qc_))
qi.partial_trace(qc_state,[0,1])

DensityMatrix([[ 0.3118348 +0.j        , -0.00761719-0.46318013j],
               [-0.00761719+0.46318013j,  0.6881652 +0.j        ]],
              dims=(2,))


<ipython-input-9-3ed9c04ba9cc>:1: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  original_qc = u_circuit.combine(qc)
